In [ ]:
import tensorflow as tf
from models import decoder, encoder, vae
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
from constants import *

In [185]:
apes_info = pd.read_csv(APES_INFO_FILEPATH)
train_ids = apes_info.loc[apes_info["dataset"] == "train", "image"].to_list()
validation_ids = apes_info.loc[apes_info["dataset"] == "validation", "image"].to_list()
test_ids = apes_info.loc[apes_info["dataset"] == "test", "image"].to_list()
images_ids = sorted([item.stem for item in pathlib.Path(DATA_FILEPATH).iterdir() if item.suffix == ".png"])

In [186]:
dataset = (
    tf.keras.utils.image_dataset_from_directory(
        directory=DATA_FILEPATH,
        batch_size=1,
        image_size=IMAGE_SIZE,
        shuffle=False,
        labels=images_ids,
    )
    .unbatch()
    .map(lambda x, y: (x / 255, y))
)


@tf.autograph.experimental.do_not_convert
def select_x(x, _):
    return x

Found 10000 files belonging to 10000 classes.


In [187]:
decoder_model = decoder.build_decoder(LATENT_DIM)
encoder_model = encoder.build_encoder(LATENT_DIM)

vae_model = vae.VAE(encoder_model, decoder_model, 100, 1)
vae_model.load_weights("../data/models/vae/")

for i in range(len(vae_model.layers)):
    vae_model.layers[i].trainable = False

In [238]:
bck = pd.concat([pd.get_dummies(apes_info["Background"]), apes_info[["dataset"]]], axis=1)
bck_train = bck[bck["dataset"] == "train"].drop(columns="dataset")
bck_val = bck[bck["dataset"] == "validation"].drop(columns="dataset")

mth = pd.concat([pd.get_dummies(apes_info["Mouth"]), apes_info[["dataset"]]], axis=1)
mth_train = mth[mth["dataset"] == "train"].drop(columns="dataset")
mth_val = mth[mth["dataset"] == "validation"].drop(columns="dataset")

In [239]:
inp = tf.keras.layers.Input((256, 256, 3))
enc = encoder_model(inp)
concat = tf.keras.layers.Concatenate()([enc[0], enc[1]])
x = tf.keras.layers.Dense(64, activation="relu")(concat)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
out1 = tf.keras.layers.Dense(8, activation="sigmoid")(x)
out2 = tf.keras.layers.Dense(33, activation="sigmoid")(x)
mod2 = tf.keras.Model(inp, [out1, out2], name="abc")

In [240]:
mod2.compile(
    optimizer="adam",
    loss=[tf.keras.losses.BinaryCrossentropy(from_logits=False), tf.keras.losses.BinaryCrossentropy(from_logits=False)],
    metrics=["accuracy"],
)

In [242]:
y_ds_train = tf.data.Dataset.zip(
    (tf.data.Dataset.from_tensor_slices(bck_train), tf.data.Dataset.from_tensor_slices(mth_train))
)
x_ds_train = dataset.filter(lambda _, y: tf.math.reduce_any(y == train_ids)).map(select_x).batch(1)
ds_train = tf.data.Dataset.zip((x_ds_train.unbatch(), y_ds_train)).batch(32)


y_ds_val = tf.data.Dataset.zip(
    (tf.data.Dataset.from_tensor_slices(bck_val), tf.data.Dataset.from_tensor_slices(mth_val))
)
x_ds_val = dataset.filter(lambda _, y: tf.math.reduce_any(y == validation_ids)).map(select_x).batch(1)
ds_val = tf.data.Dataset.zip((x_ds_val.unbatch(), y_ds_val)).batch(32)

In [244]:
mod2.fit(ds_train, validation_data=ds_val, epochs=5, batch_size=None)

Epoch 1/5


2023-05-19 18:26:16.090938: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_24' with dtype bool and shape [7000,33]
	 [[{{node Placeholder/_24}}]]
2023-05-19 18:26:16.091353: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [10000]
	 [[{{node Placeholder/_4}}]]


    219/Unknown - 31s 137ms/step - loss: 0.3857 - dense_87_loss: 0.2039 - dense_88_loss: 0.1818 - dense_87_accuracy: 0.7157 - dense_88_accuracy: 0.1931

2023-05-19 18:26:46.875719: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_24' with dtype bool and shape [1500,33]
	 [[{{node Placeholder/_24}}]]
2023-05-19 18:26:46.876013: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10000]
	 [[{{node Placeholder/_0}}]]


219/219 [==============================] - 47s 212ms/step - loss: 0.3857 - dense_87_loss: 0.2039 - dense_88_loss: 0.1818 - dense_87_accuracy: 0.7157 - dense_88_accuracy: 0.1931 - val_loss: 0.1767 - val_dense_87_loss: 0.0519 - val_dense_88_loss: 0.1249 - val_dense_87_accuracy: 0.9700 - val_dense_88_accuracy: 0.2740
Epoch 2/5
219/219 [==============================] - 46s 208ms/step - loss: 0.1426 - dense_87_loss: 0.0323 - dense_88_loss: 0.1104 - dense_87_accuracy: 0.9747 - dense_88_accuracy: 0.3097 - val_loss: 0.1337 - val_dense_87_loss: 0.0226 - val_dense_88_loss: 0.1111 - val_dense_87_accuracy: 0.9993 - val_dense_88_accuracy: 0.3753
Epoch 3/5
219/219 [==============================] - 45s 207ms/step - loss: 0.1143 - dense_87_loss: 0.0160 - dense_88_loss: 0.0984 - dense_87_accuracy: 0.9859 - dense_88_accuracy: 0.3843 - val_loss: 0.1030 - val_dense_87_loss: 0.0080 - val_dense_88_loss: 0.0950 - val_dense_87_accuracy: 1.0000 - val_dense_88_accuracy: 0.4173
Epoch 4/5
219/219 [=============

In [245]:
mod2.summary()

Model: "abc"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_25 (InputLayer)       [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 encoder (Functional)        [(None, 256),                2201184   ['input_25[0][0]']            
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                                  
 concatenate_14 (Concatenat  (None, 512)                  0         ['encoder[12][0]',            
 e)                                                                  'encoder[12][1]']          